Xây dựng lớp PrimalityTest để so sánh thời gian chạy (running time) của các thuật toán đã học:

1. Thuật toán tất định (Trial division) trang 71, định lý 3.2 trong giáo trình.

2. Thuật toán Rabin-Miller, xem lại từ trang 228 tới hết section.

3. Thuật toán Soloway – Strassen, định lý 11.18 trang 460 (lưu ý thuật toán này có cần một hàm số để tính kí hiệu Jacobi, xem thêm từ trang 448 tới định lý 11.12 để hiểu cách tính nhanh).

Lớp PrimalityTest có chức năng sau:

- Truyền vào một số nguyên dương n, nếu người dùng nhập vào số âm hay số thực thì báo lỗi nhập sai bắt nhập lại cho đến khi đúng là số nguyên dương thì thôi.

- Có chức năng kiểm tra số vừa nhập là số nguyên tố hay hợp số cho mỗi thuật toán trong các thuật toán trên, hiển thị thêm thời gian chạy.

- Có chức năng so sánh thuật toán nào chạy nhanh hơn (chẳng hạn A>B>C)

Lưu ý: Không dùng sympy. Hướng dẫn cách nhập test case.

- Case 1: n là một số nguyên tố hoặc hợp số < 1000.

- Case 2: lấy hai số nguyên tố p và q có 4 chữ số rồi nhân lại với nhau ta được n=pq

- Case 3: Lên google tìm một số nguyên tố có 7 hoặc 8 chữ số rồi nhập vào.

In [180]:
from time import perf_counter # using perf_counter is more precise than time()
import math
# from rich import print # pretty print
import random
import numpy as np
from collections.abc import Callable
from functools import wraps
from typing import Literal
from itertools import pairwise

def timing(func: Callable):
    """Output time taken of function"""

    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = perf_counter()
        result = func(*args, **kwargs)
        duration = np.format_float_positional(perf_counter() - tic)
        print(f' - {func.__name__}() took {duration}s')
        return result

    return wrapper


In [196]:
class PrimalityTest:
    def __init__(self, number) -> None:
        while not isinstance(number, int) or number < 0:
            print(f'{number} is invalid. Number must be integer positive')
            try:
                number = input('Enter number: ')
                number = int(number)
            except ValueError:
                pass

        self.number = number

    def __repr__(self):
        return str(self.number)
    
    @timing
    def is_prime(self, get_duration=False) -> bool | float:
        """Check if a number is a prime, using Trial division"""
        if get_duration:
            tic = perf_counter()
        
        if self.number == 2:
            return True

        if self.number < 2 or self.number % 2 == 0:
            return False

        i = 3
        while i < math.floor(math.sqrt(self.number)) + 1:
            if self.number % i == 0:
                if get_duration:
                    return perf_counter() - tic
                return False
            i += 2

        if get_duration:
            return perf_counter() - tic
        return True

    @timing    
    def is_composite(self) -> bool:
        """Check if a number is composite"""
        for i in range(2, self.number):
            if self.number % i == 0:
                return True

        return False

    @timing
    def rabin_miller(self, k=10, get_duration=False) -> bool | float:
        """Rabin-Miller test, return False if number is composite, 
        True if number is probably prime
        """
        
        def miller_test(b, n):
            # n - 1 = t.2^s
            t = n - 1
            while t % 2 == 0:
                t //= 2
            
            if pow(b, t, n) == 1:
                return True

            while t < n - 1:
                if pow(b, t, n) == n - 1:
                    return True
                
                t *= 2

            return False

        if get_duration:
            tic = perf_counter()
        
        if self.number < 2 or self.number % 2 == 0:
            return False
        
        # n - 1 = 2^s.t
        for _ in range(k):
            b = random.randint(1, self.number - 1)
            if not miller_test(b, self.number):
                if get_duration:
                    return perf_counter() - tic
                return False

        if get_duration:
            return perf_counter() - tic
        return True

    @timing
    def solovay_strassen(self, k=10, get_duration=False) -> bool | float:
        """Solovay Strassen test"""

        def jacobi_symbol(a, b) -> Literal[-1, 0, 1]:
            """Fast calculate Jacobi symbol"""
            if math.gcd(a, b) != 1:
                return 0
            b_list, s_list = [], []
            r = a - (a // b) * b
            while r > 1:
                r = a - (a // b) * b
                s = 0
                while r % 2 == 0:
                    s += 1
                    r //= 2
                s_list.append(s)
                b_list.append(b)
                a, b = b, r

            exponent = 0
            for s, b in zip(s_list, b_list):
                exponent += s * ((b**2 - 1) // 8)

            for x, y in list(pairwise(b_list)):
                exponent += (x-1) * (y-1) // 4
                        
            return -1 if exponent & 1 else 1
        
        if get_duration:
            tic = perf_counter()
        
        if self.number < 2 or self.number % 2 == 0:
            return False
        
        for _ in range(k):
            b = random.randint(1, self.number-1)
            j = jacobi_symbol(b, self.number)
            if j == 0 or pow(b, (self.number-1)//2, self.number) != j % self.number:
                if get_duration:
                    return perf_counter() - tic
                return False
        
        if get_duration:
            return perf_counter() - tic
        return True

    def performance(self, k=10) -> None:
        result = {}
        result['trial_division'] = self.is_prime(get_duration=True)
        result['rabin_miller'] = self.rabin_miller(k, get_duration=True)
        result['solovay_strassen'] = self.solovay_strassen(k, get_duration=True)

        result = {k: v for k, v in 
                  sorted(result.items(), key=lambda item: item[1])}
        print('=> Performance: ', end='')
        for i, (func, duration) in enumerate(result.items()):
            if i == 2:
                print(f'{func}', end='')
            else:
                print(f'{func} > ', end='')
        print()

In [182]:
n = PrimalityTest(-12)
print(f'n currently is {n.number}')

-12 is invalid. Number must be integer positive
-123 is invalid. Number must be integer positive
1.5 is invalid. Number must be integer positive
n currently is 2


In [197]:
for i in range(1, 11):
    if PrimalityTest(i).is_prime():
        print(i, 'is prime')
    else:
        print(i, 'is not prime')

 - is_prime() took 0.000004599918611347675s
1 is not prime
 - is_prime() took 0.000002600019797682762s
2 is prime
 - is_prime() took 0.000006500049494206905s
3 is prime
 - is_prime() took 0.000000600004568696022s
4 is not prime
 - is_prime() took 0.000001300009898841381s
5 is prime
 - is_prime() took 0.000000500003807246685s
6 is not prime
 - is_prime() took 0.00000100000761449337s
7 is prime
 - is_prime() took 0.000000500003807246685s
8 is not prime
 - is_prime() took 0.000001500011421740055s
9 is not prime
 - is_prime() took 0.000000700005330145359s
10 is not prime


In [198]:
for i in range(1, 11):
    if PrimalityTest(i).is_composite():
        print(i, 'is composite')
    else:
        print(i, 'is not composite')

 - is_composite() took 0.0000030999071896076202s
1 is not composite
 - is_composite() took 0.000001100008375942707s
2 is not composite
 - is_composite() took 0.000001200009137392044s
3 is not composite
 - is_composite() took 0.000000700005330145359s
4 is composite
 - is_composite() took 0.000000800006091594696s
5 is not composite
 - is_composite() took 0.000000600004568696022s
6 is composite
 - is_composite() took 0.000000800006091594696s
7 is not composite
 - is_composite() took 0.000000500003807246685s
8 is composite
 - is_composite() took 0.000000600004568696022s
9 is composite
 - is_composite() took 0.000000500003807246685s
10 is composite


## Case 1

In [199]:
# source: https://t5k.org/curios/page.php/499.html
n = PrimalityTest(499)
if n.is_prime():
    print(f'Using Trial division, {n} is prime')
else:
    print(f'Using Trial division, {n} is composite')

if n.rabin_miller():
    print(f'Using Rabin-Miller test, {n} probably prime')
else:
    print(f'Using Rabin-Miller test, {n} is composite')

if n.solovay_strassen():
    print(f'Using Solovay Strassen test, {n} probably prime')
else:
    print(f'Using Solovay Strassen test, {n} is composite')
print()
n.performance()

 - is_prime() took 0.000006900052540004253s
Using Trial division, 499 is prime
 - rabin_miller() took 0.00003450002986937761s
Using Rabin-Miller test, 499 probably prime
 - solovay_strassen() took 0.000051600043661892414s
Using Solovay Strassen test, 499 probably prime

 - is_prime() took 0.000004200031980872154s
 - rabin_miller() took 0.00002519995905458927s
 - solovay_strassen() took 0.000042499974370002747s
=> Performance: trial_division > rabin_miller > solovay_strassen


In [200]:
for n in random.sample(range(2, 1000), 5):
    n = PrimalityTest(n)
    print(f'{n = }')
    n.performance()

n = 152
 - is_prime() took 0.000002600019797682762s
 - rabin_miller() took 0.000001600012183189392s
 - solovay_strassen() took 0.000005600042641162872s
=> Performance: trial_division > rabin_miller > solovay_strassen
n = 184
 - is_prime() took 0.0000004998873919248581s
 - rabin_miller() took 0.000000500003807246685s
 - solovay_strassen() took 0.000001300009898841381s
=> Performance: trial_division > rabin_miller > solovay_strassen
n = 343
 - is_prime() took 0.000003400025889277458s
 - rabin_miller() took 0.000008999952115118504s
 - solovay_strassen() took 0.000010999967344105244s
=> Performance: trial_division > rabin_miller > solovay_strassen
n = 327
 - is_prime() took 0.00000100000761449337s
 - rabin_miller() took 0.000003400025889277458s
 - solovay_strassen() took 0.000006299931555986404s
=> Performance: trial_division > rabin_miller > solovay_strassen
n = 626
 - is_prime() took 0.000000500003807246685s
 - rabin_miller() took 0.0000004998873919248581s
 - solovay_strassen() took 0.00

## Case 2

In [201]:
def generate_random_prime():
    """Generate a prime have 4 digits"""
    def is_prime(n) -> bool:
        if n == 2:
            return True

        if n < 2 or n % 2 == 0:
            return False

        i = 3
        while i < math.floor(math.sqrt(n)) + 1:
            if n % i == 0:
                return False
            i += 2

        return True
    
    checked = set()
    while True:
        n = random.randint(1001, 9999)
        while n % 2 == 0 or n in checked:
            n = random.randint(1001, 9999)

        if is_prime(n):
            return n
        else:
            checked.add(n)

In [202]:
p = generate_random_prime()
print(f'{p = }')
q = generate_random_prime()
print(f'{q = }')
n = PrimalityTest(p * q)
print(f'{n = }')

print(n.is_prime())
print(n.rabin_miller())
print(n.solovay_strassen())
print()
n.performance()

p = 7027
q = 5861
n = 41185247
 - is_prime() took 0.0004257999826222658s
False
 - rabin_miller() took 0.00000879995059221983s
False
 - solovay_strassen() took 0.00002319994382560253s
False

 - is_prime() took 0.0004190000472590327s
 - rabin_miller() took 0.000005799927748739719s
 - solovay_strassen() took 0.000013599987141788006s
=> Performance: rabin_miller > solovay_strassen > trial_division


In [203]:
for _ in range(5):
    p = generate_random_prime()
    q = generate_random_prime()
    print(f'{p = }, {q = }', end=', ')
    n = PrimalityTest(p * q)
    print(f'{n = }')

    n.performance()
    print()

p = 6469, q = 1009, n = 6527221
 - is_prime() took 0.00008130003698170185s
 - rabin_miller() took 0.000013599987141788006s
 - solovay_strassen() took 0.000020700041204690933s
=> Performance: rabin_miller > solovay_strassen > trial_division

p = 6299, q = 5503, n = 34663397
 - is_prime() took 0.00042470009066164494s
 - rabin_miller() took 0.000007500057108700275s
 - solovay_strassen() took 0.000014899997040629387s
=> Performance: rabin_miller > solovay_strassen > trial_division

p = 4679, q = 1709, n = 7996411
 - is_prime() took 0.00021939992439001799s
 - rabin_miller() took 0.00000909995287656784s
 - solovay_strassen() took 0.00002619996666908264s
=> Performance: rabin_miller > solovay_strassen > trial_division

p = 2083, q = 6619, n = 13787377
 - is_prime() took 0.0002943000290542841s
 - rabin_miller() took 0.000020200037397444248s
 - solovay_strassen() took 0.000031100003980100155s
=> Performance: rabin_miller > solovay_strassen > trial_division

p = 3727, q = 7883, n = 29379941
 - i

## Case 3

Source: [https://t5k.org/curios/index.php?start=7&stop=8](https://t5k.org/curios/index.php?start=7&stop=8)

In [204]:
n_list = [5858581, 7576757, 10234589, 77767777, 99999989]
for n in n_list:
    print(f'{n = }')
    PrimalityTest(n).performance()

n = 5858581
 - is_prime() took 0.0001897999318316579s
 - rabin_miller() took 0.0000439999857917428s
 - solovay_strassen() took 0.0000817000400274992s
=> Performance: rabin_miller > solovay_strassen > trial_division
n = 7576757
 - is_prime() took 0.00020110001787543297s
 - rabin_miller() took 0.00003850006032735109s
 - solovay_strassen() took 0.00008219992741942406s
=> Performance: rabin_miller > solovay_strassen > trial_division
n = 10234589
 - is_prime() took 0.00040120002813637257s
 - rabin_miller() took 0.00005309993866831064s
 - solovay_strassen() took 0.00012829992920160294s
=> Performance: rabin_miller > solovay_strassen > trial_division
n = 77767777
 - is_prime() took 0.0007340000011026859s
 - rabin_miller() took 0.00007219996768981218s
 - solovay_strassen() took 0.00008060003165155649s
=> Performance: rabin_miller > solovay_strassen > trial_division
n = 99999989
 - is_prime() took 0.0007313999813050032s
 - rabin_miller() took 0.0000347999157384038s
 - solovay_strassen() took 0.

In [205]:
PrimalityTest(99999999977).performance()

 - is_prime() took 0.033713100012391806s
 - rabin_miller() took 0.00017419992946088314s
 - solovay_strassen() took 0.00021049997303634882s
=> Performance: rabin_miller > solovay_strassen > trial_division


In [206]:
PrimalityTest(1000000000100011).performance()

 - is_prime() took 3.2682928000576794s
 - rabin_miller() took 0.00013529998250305653s
 - solovay_strassen() took 0.0003160999622195959s
=> Performance: rabin_miller > solovay_strassen > trial_division
